# Profitable Strategy with Markov Chain

In the excel sheet, I've proven that the change in price in each days are not random. So using this information, I made a simple strategy based on single transition and buying selling on opening and closing on the same day. Now let's try it for multiple transition and test it as well.

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import scipy.stats as stats
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_raw = pd.read_csv('AMD.csv')
data_raw['Date'] = pd.to_datetime(data_raw['Date'])
data_raw 

,Date,Open,High,Low,Close,Adj Close,Volume
0,1980-03-18,0.000000,3.125000,2.937500,3.031250,3.031250,727200
1,1980-03-19,0.000000,3.083333,3.020833,3.041667,3.041667,295200
2,1980-03-20,0.000000,3.062500,3.010417,3.010417,3.010417,159600
3,1980-03-21,0.000000,3.020833,2.906250,2.916667,2.916667,130800
4,1980-03-24,0.000000,2.916667,2.635417,2.666667,2.666667,436800
...,...,...,...,...,...,...,...
11026,2023-12-11,130.559998,135.039993,128.250000,134.410004,134.410004,98668300
11027,2023-12-12,132.250000,139.889999,131.899994,137.610001,137.610001,111677800
11028,2023-12-13,137.970001,139.240005,136.029999,138.190002,138.190002,70651600
11029,2023-12-14,138.889999,141.820007,135.729996,138.000000,138.000000,76115200


Split training and test data set. Set the test as the recent 3 years, and train our strategy on all data before that.  

In [3]:
data_test = data_raw [data_raw["Date"]>'2020-12-14']
data = data_raw [data_raw["Date"]<'2020-12-15']

# Data cleaning & Feature Engineering

Get Percentage daily change of price but keep it decimal

In [4]:
data["DailyChange"] = (data["Close"]- data["Open"])/data["Open"]
data

,Date,Open,High,Low,Close,Adj Close,Volume,DailyChange
0,1980-03-18,0.000000,3.125000,2.937500,3.031250,3.031250,727200,inf
1,1980-03-19,0.000000,3.083333,3.020833,3.041667,3.041667,295200,inf
2,1980-03-20,0.000000,3.062500,3.010417,3.010417,3.010417,159600,inf
3,1980-03-21,0.000000,3.020833,2.906250,2.916667,2.916667,130800,inf
4,1980-03-24,0.000000,2.916667,2.635417,2.666667,2.666667,436800,inf
...,...,...,...,...,...,...,...,...
10270,2020-12-08,94.050003,94.739998,91.900002,92.919998,92.919998,33907500,-0.012015
10271,2020-12-09,92.760002,94.699997,89.160004,89.830002,89.830002,52170400,-0.031587
10272,2020-12-10,89.550003,92.089996,89.029999,91.660004,91.660004,33804400,0.023562
10273,2020-12-11,91.540001,92.330002,90.160004,91.650002,91.650002,28368100,0.001202


Remove invalid values and infinite daily returns due to error of opening being 0.

In [5]:
data = data.dropna()
data = data[data["Open"]!=0]
data = data[1:]
data = data.drop(data.columns[[1,2,3,4,5,6]],axis = 1)
data

,Date,DailyChange
760,1983-03-21,0.003484
761,1983-03-22,-0.025090
762,1983-03-23,0.025735
763,1983-03-24,0.007168
764,1983-03-25,-0.003509
...,...,...
10270,2020-12-08,-0.012015
10271,2020-12-09,-0.031587
10272,2020-12-10,0.023562
10273,2020-12-11,0.001202


Let's add labels to each of the levels of daily change.

In [6]:
labels = ["E","D","C","B","A"]
data["CurrentLabel"] = pd.qcut(data["DailyChange"], 5, labels=labels)
data #A indicates a relatively high return and E is a high loss or low return. 

,Date,DailyChange,CurrentLabel
760,1983-03-21,0.003484,C
761,1983-03-22,-0.025090,E
762,1983-03-23,0.025735,A
763,1983-03-24,0.007168,B
764,1983-03-25,-0.003509,C
...,...,...,...
10270,2020-12-08,-0.012015,D
10271,2020-12-09,-0.031587,E
10272,2020-12-10,0.023562,A
10273,2020-12-11,0.001202,C


Below are ranges and details of what each labels represent in returns. In summary, A is a high return, C is a no return, E is a high loss situation.

In [7]:
data[data["CurrentLabel"]=="A"]["DailyChange"].describe()

count    1903.000000
mean        0.043905
std         0.025670
min         0.020906
25%         0.027093
50%         0.035821
75%         0.052348
max         0.273885
Name: DailyChange, dtype: float64

In [8]:
data[data["CurrentLabel"]=="B"]["DailyChange"].describe()

count    1903.000000
mean        0.012142
std         0.004573
min         0.004464
25%         0.008000
50%         0.012048
75%         0.015748
max         0.020896
Name: DailyChange, dtype: float64

In [9]:
data[data["CurrentLabel"]=="C"]["DailyChange"].describe()

count    1901.000000
mean       -0.001611
std         0.003410
min        -0.007937
25%        -0.004545
50%         0.000000
75%         0.000000
max         0.004459
Name: DailyChange, dtype: float64

In [10]:
data[data["CurrentLabel"]=="D"]["DailyChange"].describe()

count    1905.000000
mean       -0.014749
std         0.004112
min        -0.022571
25%        -0.018085
50%        -0.014286
75%        -0.011416
max        -0.007937
Name: DailyChange, dtype: float64

In [11]:
data[data["CurrentLabel"]=="E"]["DailyChange"].describe()

count    1903.000000
mean       -0.041666
std         0.020431
min        -0.281250
25%        -0.048258
50%        -0.035573
75%        -0.028302
max        -0.022581
Name: DailyChange, dtype: float64

Now let's add the columns with history of the labels. 

In [12]:

data["CurrentLabel"] = data["CurrentLabel"].astype("str")
data["1dayLabel"] =  data["CurrentLabel"].shift(1)
for i in range(2,6):
    col_name = str(i) + "dayLabel"
    prev_col_name = str(i-1) + "dayLabel"
    data[col_name] =  data["CurrentLabel"].shift(i) +  data[prev_col_name]
data["Next_day"] = data["CurrentLabel"].shift(-1)
data


,Date,DailyChange,CurrentLabel,1dayLabel,2dayLabel,3dayLabel,4dayLabel,5dayLabel,Next_day
760,1983-03-21,0.003484,C,None,NaN,NaN,NaN,NaN,E
761,1983-03-22,-0.025090,E,C,NaN,NaN,NaN,NaN,A
762,1983-03-23,0.025735,A,E,CE,NaN,NaN,NaN,B
763,1983-03-24,0.007168,B,A,EA,CEA,NaN,NaN,C
764,1983-03-25,-0.003509,C,B,AB,EAB,CEAB,NaN,D
...,...,...,...,...,...,...,...,...,...
10270,2020-12-08,-0.012015,D,D,BD,DBD,BDBD,CBDBD,E
10271,2020-12-09,-0.031587,E,D,DD,BDD,DBDD,BDBDD,A
10272,2020-12-10,0.023562,A,E,DE,DDE,BDDE,DBDDE,C
10273,2020-12-11,0.001202,C,A,EA,DEA,DDEA,BDDEA,A


Now let's analyse the 2 day sequence of returns. First create table of count for what return follows a combination of 2 sequence of returns.

In [13]:
two_day_outcome = data.groupby(["2dayLabel","Next_day"])["Next_day"].count()
two_day_outcome

2dayLabel  Next_day
AA         A            97
           B            62
           C            77
           D            84
           E           104
                      ... 
EE         A           107
           B            76
           C            67
           D            75
           E           124
Name: Next_day, Length: 125, dtype: int64

Since we are looking for a strategy with high returns, let's look at what is mostly likely to have outcome A (Relatively high daily return) or B (a return) in the next day. 

In [14]:
result = pd.DataFrame(columns = ["Label","outcome_no"])

for i in range(len(two_day_outcome)):
    if two_day_outcome.keys()[i][1] == "A" or two_day_outcome.keys()[i][1] == "B":
        result.loc[len(result)] = [two_day_outcome.keys()[i],two_day_outcome[i]]
result = result.sort_values("outcome_no", ascending = False).reset_index(drop="True")
result.head()

,Label,outcome_no
0,"(EE, A)",107
1,"(CD, B)",107
2,"(CC, B)",98
3,"(AE, A)",98
4,"(BC, B)",97


A high gains day is most likely to occur after 2 heavily lost days. For simplicity, we will use top 3 scenarios that result in a high day in our strategy. So we'll only buy and sell on days after EE or CD or CC has occurred.

## Testing the 2 day sequence

Add the same features in the testing set of recent 3 years we separated earlier.

In [15]:
data_test["Date"] = pd.to_datetime(data_test["Date"],dayfirst= True)
data_test["DailyChange"] = (data_test["Close"]- data_test["Open"])/data_test["Open"]
data_test = data_test.drop(data_test.columns[[1,2,3,4,5,6]],axis = 1).reset_index(drop = "True")
data_test

,Date,DailyChange
0,2020-12-15,0.012405
1,2020-12-16,-0.001958
2,2020-12-17,-0.007278
3,2020-12-18,-0.013879
4,2020-12-21,-0.010822
...,...,...
751,2023-12-11,0.029488
752,2023-12-12,0.040529
753,2023-12-13,0.001595
754,2023-12-14,-0.006408


In [16]:
# applies the A,B,C,D,E range we had in our training set to the testing set. 
def return_classify(data,a_min,b_min,c_min,d_min):
    
    if data["DailyChange"] >= a_min:
        return "A"
    elif data["DailyChange"] >= b_min:
        return "B"
    elif data["DailyChange"] >= c_min:
        return "C"
    elif data["DailyChange"] >= d_min:
        return "D"
    else:
        return "E"

In [17]:
a_min = data[data["CurrentLabel"]=="A"]["DailyChange"].min()
b_min = data[data["CurrentLabel"]=="B"]["DailyChange"].min()
c_min = data[data["CurrentLabel"]=="C"]["DailyChange"].min()
d_min = data[data["CurrentLabel"]=="D"]["DailyChange"].min()

data_test["CurrentLabel"] = data_test.apply(return_classify,args = (a_min,b_min,c_min,d_min),axis = 1)
data_test["1dayLabel"] = data_test["CurrentLabel"].shift(1)
for i in range(2,6):
    col_name = str(i) + "dayLabel"
    prev_col_name = str(i-1) + "dayLabel"
    data_test[col_name] =   data_test["CurrentLabel"].shift(i) + data_test[prev_col_name]
data_test["Next_day"] = data_test["CurrentLabel"].shift(-1)
data_test

,Date,DailyChange,CurrentLabel,1dayLabel,2dayLabel,3dayLabel,4dayLabel,5dayLabel,Next_day
0,2020-12-15,0.012405,B,None,NaN,NaN,NaN,NaN,C
1,2020-12-16,-0.001958,C,B,NaN,NaN,NaN,NaN,C
2,2020-12-17,-0.007278,C,C,BC,NaN,NaN,NaN,D
3,2020-12-18,-0.013879,D,C,CC,BCC,NaN,NaN,D
4,2020-12-21,-0.010822,D,D,CD,CCD,BCCD,NaN,C
...,...,...,...,...,...,...,...,...,...
751,2023-12-11,0.029488,A,C,AC,EAC,BEAC,DBEAC,A
752,2023-12-12,0.040529,A,A,CA,ACA,EACA,BEACA,C
753,2023-12-13,0.001595,C,A,AA,CAA,ACAA,EACAA,C
754,2023-12-14,-0.006408,C,C,AC,AAC,CAAC,ACAAC,C


Now let's test out our strategy and see how much they make on average per year.

In [18]:
strategy = data_test[(data_test["2dayLabel"]== "EE") | 
                     (data_test["2dayLabel"]== "CD") | 
                     (data_test["2dayLabel"]== "CC") ]
strategy

,Date,DailyChange,CurrentLabel,1dayLabel,2dayLabel,3dayLabel,4dayLabel,5dayLabel,Next_day
3,2020-12-18,-0.013879,D,C,CC,BCC,NaN,NaN,D
4,2020-12-21,-0.010822,D,D,CD,CCD,BCCD,NaN,C
7,2020-12-24,0.000109,C,D,CD,DCD,DDCD,CDDCD,D
9,2020-12-29,-0.011346,D,D,CD,DCD,CDCD,DCDCD,B
13,2021-01-05,0.007275,B,C,CC,BCC,DBCC,DDBCC,D
...,...,...,...,...,...,...,...,...,...
731,2023-11-10,0.037079,A,C,CC,BCC,CBCC,ACBCC,D
735,2023-11-16,0.022440,A,D,CD,DCD,ADCD,CADCD,B
744,2023-11-30,-0.019820,D,C,CC,BCC,CBCC,ACBCC,B
745,2023-12-01,0.012596,B,D,CD,CCD,BCCD,CBCCD,D


In [19]:
market_return = round(((1 + data_test["DailyChange"]).prod()**(1/3)-1)*100,2)
default_average_return = round(np.mean(data_test["DailyChange"])**100,2)
strategy_return = round(((1 + strategy["DailyChange"]).prod()**(1/3)-1)*100,2)
strategy_average_return = round(np.mean(strategy["DailyChange"])*100,2)

print(f"Simple buy and sell everyday Yearly Return: {market_return}%")
print(f"Simple buy and sell everyday Daily Average Return: {default_average_return}%")
print(f"Strategy Yearly Return: {strategy_return}%")
print(f"Strategy Daily Average Return: {strategy_average_return}%")

Simple buy and sell everyday Yearly Return: 1.83%
Simple buy and sell everyday Daily Average Return: 0.0%
Strategy Yearly Return: -4.75%
Strategy Daily Average Return: -0.12%


Our strategy actually does worse than buying and selling everyday. Now let's try more number of transitions.

In [20]:
def strategy_compare(data,data_test):
    final_result = pd.DataFrame(columns = ["transition_no","yearly_return","risk","count"])
    final_result.loc[len(final_result)] = ["Default",(1 + data_test["DailyChange"]).prod()**(1/3)-1,np.std(data_test["DailyChange"]),len(data_test)]
    for i in range(5):
        col_name = str(i+1)+"dayLabel"
            
        outcome = data.groupby([col_name,"Next_day"])["Next_day"].count()
        result = pd.DataFrame(columns = ["Label","outcome_no"])
        #get everything that has outcomes A or B
        for j in range(len(outcome)):
            if outcome.keys()[j][1] == "A" or outcome.keys()[j][1] == "B":
                result.loc[len(result)] = [outcome.keys()[j],outcome[j]]
        result = result.sort_values("outcome_no", ascending = False).reset_index(drop=True)
        #get the top three strategies and put the result into the final_result table
        labels = [result["Label"][0][0],result["Label"][1][0],result["Label"][2][0]]
        print(f"Strategies {i+1}: {labels}")
        strategy = data_test[data_test[col_name].isin(labels)]
        final_result.loc[len(final_result)] = [col_name,(1 + strategy["DailyChange"]).prod()**(1/3)-1,np.std(strategy["DailyChange"]),len(strategy)]
        
    return final_result
    

In [21]:
strategy_compare(data,data_test)

Strategies 1: ['E', 'D', 'A']
Strategies 2: ['EE', 'CD', 'CC']
Strategies 3: ['CCD', 'BDD', 'EAE']
Strategies 4: ['CCCD', 'BDDC', 'DCCD']
Strategies 5: ['AACCA', 'BEAAA', 'EAEAE']


,transition_no,yearly_return,risk,count
0,Default,0.018309,0.027557,756
1,1dayLabel,0.117549,0.029361,425
2,2dayLabel,-0.047463,0.026304,95
3,3dayLabel,0.017496,0.026153,17
4,4dayLabel,0.018056,0.018501,4
5,5dayLabel,0.000597,0.000000,1


Except for looking back 1 day, the strategy doesn't seem to really work. Let's dig deeper to see what's working and what's causing the return to be low. Let's first look at how our test data is distributed then look at our best and the worst cases: 1day and 2day.

In [22]:
#Test data's data
default = data_test.groupby(["CurrentLabel"])["CurrentLabel"].count()
default=pd.DataFrame(default)
total = default["CurrentLabel"].sum()
default["Proportion"] = default["CurrentLabel"]/total
default

,CurrentLabel,Proportion
CurrentLabel,,
A,153,0.202381
B,162,0.214286
C,169,0.223545
D,145,0.191799
E,127,0.167989


About 64% of the test data is within A,B and C. Thus this was able to offset the relatively lesser days on D and E and generate 1.8% average annual return. 

In [23]:
# 1day
col_name = "1dayLabel"
outcome = data.groupby([col_name,"Next_day"])["Next_day"].count()
result = pd.DataFrame(columns = ["Label","outcome_no"])
#get everything that has outcomes A or B
for j in range(len(outcome)):
    if outcome.keys()[j][1] == "A" or outcome.keys()[j][1] == "B":
        result.loc[len(result)] = [outcome.keys()[j],outcome[j]]
result = result.sort_values("outcome_no", ascending = False).reset_index(drop=True)
#get the top three strategies and put the result into the final_result table
labels = [result["Label"][0][0],result["Label"][1][0],result["Label"][2][0]]
print(f"Strategies: {labels}")
strategy = data_test[data_test[col_name].isin(labels)]
one_day = strategy.groupby(["CurrentLabel"])["CurrentLabel"].count()
one_day=pd.DataFrame(one_day)
total = one_day["CurrentLabel"].sum()
one_day["Proportion"] = one_day["CurrentLabel"]/total
one_day


Strategies: ['E', 'D', 'A']


,CurrentLabel,Proportion
CurrentLabel,,
A,93,0.218824
B,97,0.228235
C,84,0.197647
D,79,0.185882
E,72,0.169412


About 64.5% of the strategy trades on A,B and C. It is of similar proportion but this strategy's A and B proportion is 44.7% vs 41.6% in the entire test data set. This is what made the 10% average yearly return difference. 

In [24]:
# 2day
col_name = "2dayLabel"
outcome = data.groupby([col_name,"Next_day"])["Next_day"].count()
result = pd.DataFrame(columns = ["Label","outcome_no"])
#get everything that has outcomes A or B
for j in range(len(outcome)):
    if outcome.keys()[j][1] == "A" or outcome.keys()[j][1] == "B":
        result.loc[len(result)] = [outcome.keys()[j],outcome[j]]
result = result.sort_values("outcome_no", ascending = False).reset_index(drop=True)
#get the top three strategies and put the result into the final_result table
labels = [result["Label"][0][0],result["Label"][1][0],result["Label"][2][0]]
print(f"Strategies: {labels}")
strategy = data_test[data_test[col_name].isin(labels)]
two_day= strategy.groupby(["CurrentLabel"])["CurrentLabel"].count()
two_day=pd.DataFrame(two_day)
total = two_day["CurrentLabel"].sum()
two_day["Proportion"] = two_day["CurrentLabel"]/total
two_day


Strategies: ['EE', 'CD', 'CC']


,CurrentLabel,Proportion
CurrentLabel,,
A,21,0.221053
B,17,0.178947
C,18,0.189474
D,24,0.252632
E,15,0.157895


Only 58.9% of this strategy consists of A,B and C which is much lower than the test data set. This would be the reason for its low return.

# Conclusion

This strategy made very little difference in choosing what days would be a profitable day. Most of these strategies made returns worse than buying and selling everyday. Although looking back 1 day strategy managed to beat the default method, it was a miniscule difference that caused this. Maybe this small difference is all we need from this strategy or maybe it's an accident that just happens for this case. In the next notebook, I'll apply the same strategy to different stocks to see if the strategy is meaningful.